# Databricks RAGアプリチュートリアル - 3. Webアプリケーションのデプロイ

このノートブックでは、前回のワークショップで作成・デプロイしたRAGエージェントを活用して、**Databricks Apps**を使用したWebアプリケーションのデプロイ手順を説明します。

## このノートブックで学習する内容
1. **Databricks Apps**の概要と利点
2. **サンプルアプリケーション**の構成理解
3. **Databricks UI**を使用したアプリ作成・デプロイ手順
4. **エージェント統合**の設定方法
5. **アプリケーションのテスト**とトラブルシューティング

## 前提条件
- 2_RAGエージェントの構築.ipynbでエージェントがModel Servingエンドポイントとしてデプロイ済みであること
- エージェントのエンドポイント名を把握していること（例：`skato_rag_workshop_tool_calling_rag_agent`）

---

## Databricks Appsとは

Databricks Appsは、データサイエンスチームが機械学習モデルやAIエージェントを活用したWebアプリケーションを簡単に構築・デプロイできるマネージドサービスです。

### 主な特徴
- **簡単デプロイ**: PythonのWebフレームワーク（Streamlit、Dash、Flask等）をサポート
- **統合認証**: Databricksワークスペースのユーザー認証と自動統合
- **スケーラブル**: トラフィックに応じた自動スケーリング
- **セキュア**: Unity Catalogとの統合によるデータガバナンス
- **モニタリング**: アプリケーションのパフォーマンス監視

---

## 3-1. アプリケーションファイルの理解

### サンプルアプリケーションの構成

このプロジェクトの`streamlit_chatbot_sample/`ディレクトリには、RAGエージェントと統合するStreamlitベースのチャットボットアプリケーションが含まれています。

```
streamlit_chatbot_sample/
├── app.py                    # メインのStreamlitアプリケーション
├── app.yaml                  # Databricks Apps設定ファイル
├── requirements.txt          # Python依存関係
└── model_serving_utils.py    # モデルサービング接続ユーティリティ
```

### 各ファイルの詳細

#### 1. `app.py` - メインアプリケーション
```python
# 主要な機能
- Streamlitベースのチャットインターフェース
- セッション状態管理による会話履歴の保持
- 環境変数SERVING_ENDPOINTからエージェントエンドポイント名を取得
- model_serving_utils.pyを使用してエージェントAPIを呼び出し
```

#### 2. `app.yaml` - Databricks Apps設定
```yaml
command: [
  "streamlit", 
  "run",
  "app.py"
]

env:
  - name: STREAMLIT_BROWSER_GATHER_USAGE_STATS
    value: "false"
  - name: "SERVING_ENDPOINT"
    valueFrom: "serving_endpoint"  # エンドポイント名をここで指定
```

**重要**: `SERVING_ENDPOINT`の値は、前のノートブックでデプロイしたエージェントのエンドポイント名に更新する必要があります。

#### 3. `requirements.txt` - 依存関係
```
mlflow>=2.21.2
streamlit==1.44.1
```

#### 4. `model_serving_utils.py` - API接続ユーティリティ
- MLflowの`get_deploy_client`を使用してDatabricksサービングエンドポイントに接続
- チャットメッセージ形式でエージェントとやり取り
- エラーハンドリングとレスポンス形式の標準化

---


## 3-2. エージェントエンドポイントの確認

### 前のノートブックでデプロイしたエージェントエンドポイント名の確認

アプリケーションをデプロイする前に、前のノートブック（2_RAGエージェントの構築）でデプロイしたエージェントのエンドポイント名を確認する必要があります。

#### エンドポイント名の確認方法

1. **Databricks UIでの確認**
   - サイドバーから「**Serving**」を選択
   - デプロイされたエンドポイントのリストを確認
   - エンドポイント名をメモしてください（例：`skato_rag_workshop_tool_calling_rag_agent`）

2. **エンドポイント名の形式**
   ```
   形式: {catalog}_{schema}_{model_name}
   例: skato_rag_workshop_tool_calling_rag_agent
   ```

#### エンドポイントの状態確認

アプリをデプロイする前に、エンドポイントが以下の状態になっていることを確認してください：

- ✅ **ステータス**: `Ready`
- ✅ **型**: `Generative AI Agent`
- ✅ **モデルバージョン**: 最新版が正常にデプロイされている

#### app.yamlファイルの更新準備

次のステップで、確認したエンドポイント名を`app.yaml`ファイルで設定します。
---



## 3-3. Databricks Apps UIでのアプリ作成

### Databricks UIでのアプリ作成手順

1. **Databricks UIへアクセス**
   - Databricksワークスペースにログイン
   - サイドバーから「**コンピュート**」を選択
   - メニューから「アプリ」を選択し、アプリの一覧を表示

2. **新しいアプリの作成**
   - 「**アプリを作成**」ボタンをクリック
   - アプリ作成ダイアログが表示されます

3. **アプリの基本設定**
   - 「**カスタムアプリを作成**」をクリック
   - アプリ名と説明を書下記のように入力し、「**次へ：構成」をクリック
      - **App name**: `<username>-rag-chatbot-app`（または任意の名前）
      - **Description**: `RAGエージェントを活用したチャットボットアプリケーション`
   - 「**リソースを追加**」をクリックし、**サービングエンドポイント**を選択。表示された項目を下記のように選択する
      - **サービングエンドポイント**: 2_RAGエージェントの構築で作成したサービングエンドポイント名
      - **権限**:`クエリー可能`を指定
      - **リソースキー**: `serving_endpoint`を指定
   - 「**アプリを作成**」をクリック

4. **ソースコードの場所指定**
   - 「**デプロイ**」ボタンを押下
   - ソースコードのパスの選択画面で、プロジェクトディレクトリ内の`streamlit_chatbot_sample`フォルダを選択
   - 「***Deploy**」をクリックして確定、デプロイ開始

5. **デプロイログの確認**

   - **ログタブの確認**
      - 「**Logs**」タブをクリック
      - デプロイとランタイムログを確認
   - **正常性確認ポイント**
   ```
   ✅ 依存関係のインストール成功
   ✅ Streamlitアプリケーションの起動
   ✅ SERVING_ENDPOINT環境変数の読み込み
   ✅ エラーメッセージがないこと
   ```
   - **デプロイ完了の確認**
      - デプロイが正常に完了すると：
         - ステータスが「**Ready**」に変わります
         - アプリケーションのURLが生成されます
         - 「**Open app**」ボタンが有効になります

## 3-4. アプリケーションのテスト

### アプリケーションへのアクセス

#### アプリの起動

1. **アプリを開く**
   - デプロイ完了後、「**Open app**」ボタンをクリック
   - 新しいタブでStreamlitアプリケーションが開きます

2. **初期画面の確認**
   - アプリのタイトル: "🧱 Chatbot App"
   - 説明文とドキュメントリンクが表示されます
   - チャット入力欄が下部に表示されます

#### RAGエージェントとの対話テスト

以下のテストケースでエージェントの動作を確認してください：

1. **基本動作テスト**
   ```
   質問: "こんにちは！"
   期待: エージェントからの挨拶応答
   ```

2. **Vector Search機能テスト**
   ```
   質問: "生成AIの開発ワークフローについて教えてください"
   期待: Vector Searchで検索された文書に基づく詳細な回答
   ```

3. **Tool-calling機能テスト**
   ```
   質問: "MLOpsのベストプラクティスについて詳しく説明してください"
   期待: 複数のツールを組み合わせた包括的な回答
   ```

4. **マルチターン会話テスト**
   ```
   1回目: "RAGシステムとは何ですか？"
   2回目: "先ほどの回答について、実装時の注意点を教えてください"
   期待: 会話履歴を踏まえた文脈的な応答
   ```

## 3-5. トラブルシューティング

### よくある問題と解決方法

#### デプロイに関する問題

##### 問題: デプロイが失敗する
**症状**: Build/Starting段階でエラーが発生
**解決方法**:
1. **ログの確認**
   - 「Logs」タブでエラーメッセージを確認
   - 依存関係のインストールエラーをチェック

2. **requirements.txtの確認**
   ```
   # 正しい形式
   mlflow>=2.21.2
   streamlit==1.44.1
   ```

3. **app.yamlの構文確認**
   - YAMLファイルのインデントが正しいことを確認
   - コマンド形式が適切であることを確認

#### エージェント接続の問題

##### 問題: エージェントからの応答がない
**症状**: "Loading..."が続く、またはエラーメッセージ
**解決方法**:
1. **エンドポイントの状態確認**
   - Servingページでエンドポイントが「Ready」状態であることを確認
   - エンドポイントのAPIテストを実行

2. **権限の確認**
   - Unity Catalogモデルへのアクセス権限を確認
   - Serving エンドポイントの実行権限を確認

##### 問題: エラーメッセージが表示される
**症状**: "This app can only run against..." エラー
**解決方法**:
1. **エンドポイントタイプの確認**
   - エンドポイントがChat Agent形式であることを確認
   - Foundation ModelエンドポイントとAgent エンドポイントの違いを理解

#### パフォーマンスの問題

##### 問題: 応答が非常に遅い
**解決方法**:
1. **エンドポイントのスケーリング設定確認**
2. **Vector Searchインデックスのパフォーマンス確認**
3. **ネットワーク遅延の確認**

#### デバッグのベストプラクティス

1. **段階的確認**
   - エンドポイントの個別テスト（Serving UI）
   - AI Playgroundでのテスト
   - アプリケーションでのテスト

2. **ログの活用**
   - アプリケーションログの定期的な確認
   - エラーパターンの把握

3. **設定の再確認**
   - app.yamlファイルの設定
   - 環境変数の値
   - エンドポイント名の正確性

## まとめ

### 🎉 **チュートリアル完了おめでとうございます！**

このチュートリアルを通じて、以下を実現しました：

#### ✅ **達成事項**

1. **Vector Search Index構築** (ノートブック1)
   - PDFドキュメントの解析と処理
   - 埋め込みベクトルの生成とインデックス化
   - 検索機能の実装

2. **Tool-calling RAGエージェント構築** (ノートブック2)
   - LangGraphによる高度なワークフロー設計
   - Unity Catalogとの統合
   - MLflowによるモデル管理とデプロイ

3. **Webアプリケーションデプロイ** (ノートブック3)
   - Databricks Appsによるユーザーフレンドリーなインターフェース
   - エンドツーエンドのRAGソリューション完成

#### 🚀 **構築したシステムの価値**

- **企業レベルの生成AI**: セキュリティとガバナンスを重視した設計
- **スケーラブル**: 大量のドキュメントと多数のユーザーに対応
- **拡張可能**: 新しいツールやデータソースの簡単な追加
- **運用性**: MLflowによる継続的な品質管理と改善

#### 📋 **次のステップの提案**

1. **品質向上**
   - 社内フィードバックの収集と反映
   - A/Bテストによる機能改善

2. **機能拡張**
   - 多言語対応
   - 音声入出力機能
   - 外部システム連携

3. **運用強化**
   - 自動化されたCI/CD pipeline
   - 本格的な監視・アラート体制

#### 🔗 **参考リンク**

- [Databricks Generative AI](https://learn.microsoft.com/azure/databricks/generative-ai/)
- [Mosaic AI Agent Framework](https://learn.microsoft.com/azure/databricks/generative-ai/agent-framework/)
- [Databricks Apps](https://docs.databricks.com/aws/ja/dev-tools/databricks-apps/)
- [Vector Search](https://learn.microsoft.com/azure/databricks/generative-ai/vector-search/)
